In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord as coord
import astropy.units as u
from astropy.coordinates import Angle
import warnings
warnings.filterwarnings('ignore')

###  Step 1:  Importing gamma ray burst data

<font color = 'purple'>Data I have used for gamma ray burst was taken form this site below.</font>

https://swift.gsfc.nasa.gov/results/batgrbcat/summary_cflux/summary_general_info/summary_general.txt


In [2]:
column_names = ['GRBname', 'RA_ground', 'DEC_ground']
df_grb = pd.read_csv('/home/astroguy/frb_summerresearch/grb_data_copy.csv', delimiter='|', header = 0)
df_grb.tail(1)

,GRBname,Trig_ID,Trig_time_met,Trig_time_UTC,RA_ground,DEC_ground,Image_position_err,Image_SNR,T90,T90_err,T50,T50_err,Evt_start_sincetrig,Evt_stop_sincetrig,pcode,Trigger_method,XRT_detection,"comment,,,"
1390,GRB041217,100116,124961305.236,2004-12-17T07:28:25.236000,1.647929e+02,-1.794047e+01,1.374749e+00,1.930439e+01,5.668000e+00,7.297123e-01,2.700000e+00,2.512051e-01,-2,18,0.2969,rate trigger,No,",,,"


<font color = 'purple'>Note:  According to the description in the catalog RA_ground and DEC_ground both has been measured in the unit of degree.</font> 

In [3]:
# Taking only necessary subset of the whold data from the catalog

df_grb_subset = df_grb.iloc[:,[0,4,5]]
print(df_grb_subset.shape)
df_grb_subset.tail(2)

(1391, 3)


,GRBname,RA_ground,DEC_ground
1389,GRB041219A,N/A,N/A
1390,GRB041217,1.647929e+02,-1.794047e+01


In [5]:
# Since some data are missing we want to get rid of those rows. Using for loop to pick the rows with no data.


empty_row_num = []
for num in np.arange(0, len(df_grb_subset.iloc[:])):
    if df_grb_subset.iloc[num, 1] == ' N/A ' or df_grb_subset.iloc[num, 2] == ' N/A ':
        empty_row_num.append(num)

empty_row_num

[222, 271, 325, 430, 483, 786, 1110, 1188, 1283, 1389]

In [6]:
# Using drop function to drop all the rows with no data. Ten rows with no data have been removed.

df_grb_subset.drop(empty_row_num, axis = 0, inplace = True)
print(df_grb_subset.shape)

(1381, 3)


###  Step 2: Working with fast radio burst data

<font color = 'purple'>Data taken from link below</font>

https://www.herta-experiment.org/frbstats/catalogue

In [7]:
columns = ['ra', 'dec', 'dm', 'flux', 'redshift', 'frb', 'l', 'b', 'width', 'fluence']
df_herta = pd.read_csv('herta_frb_catalogue.csv', usecols=columns).replace('-', '0')
df_herta.head(1)

,frb,ra,dec,l,b,dm,flux,width,fluence,redshift
0,FRB 20010125A,19:06:53,-40:37:14,356.64,-20.02,790.3,0.54,0,0,0.7083


<font color = 'purple'>Unit of RA in the catalog is hour:min:sec and that of declination is in deg:arcmin:arcsec</font>

In [8]:
herta_subset = df_herta.iloc[:, [0, 1,2]]
herta_subset.head(1)

,frb,ra,dec
0,FRB 20010125A,19:06:53,-40:37:14


<font color = 'purple'>We convert RA and DEC to the unit of degree to make it uniform with the unit of RA and DEC in GRB data. We use astropy library to convert coordinates to degress. </font>

Reference:  https://docs.astropy.org/en/stable/coordinates/angles.html

<font color = 'purple'>This site maintained by penstate can be used as checker to see if our conversion has been done correctly.</font>

https://www.swift.psu.edu/secure/toop/convert.htm

In [9]:
RA_in_degrees  = []
DEC_in_degrees = []

for k in np.arange(0, len(herta_subset['frb'])):
    RA_degree = Angle(herta_subset['ra'][k], unit = 'hourangle')   # RA is in  hr:min:sec
    DEC_degree = Angle(herta_subset['dec'][k], unit = 'degree')    # DEC is in deg:arcmin:arcsec
    RA_in_degrees.append(RA_degree.degree)
    DEC_in_degrees.append(DEC_degree.degree)
    
herta_subset['RA_in_deg'] = RA_in_degrees
herta_subset['DEC_in_deg'] = DEC_in_degrees

herta_subset.head(1)

,frb,ra,dec,RA_in_deg,DEC_in_deg
0,FRB 20010125A,19:06:53,-40:37:14,286.720833,-40.620556


In [10]:
# This function calculates distance using formula shown below.

def distance(ra_frb, ra_grb, dec_frb, dec_grb):
    diff_ra = ra_frb - ra_grb
    diff_dec = dec_frb - dec_grb
    avg_dec = (dec_frb + dec_grb)/2
    frb_grb_dist = np.sqrt((diff_ra*np.cos(avg_dec*(np.pi/180)))**2 + diff_dec**2)
    return frb_grb_dist

$$ dist = \sqrt{ ({\Delta\alpha\cos{\delta}})^2 + {\Delta{\delta}}^2} $$

where, 

<br>$ \Delta \alpha = ra_{1} - ra_{2} $
<br>$\Delta \delta = de_{1} - de_{2} $ 
<br>$ \delta = \frac{de_{1} + de_{2}}{2}$

In [11]:
dist_s_no = []
for m in np.arange(0, len(herta_subset['frb'])):
    frb_ra = herta_subset['RA_in_deg'][m]
    frb_dec = herta_subset['DEC_in_deg'][m]

    dist_list = []
    for n in np.arange(0, len(df_grb_subset.iloc[:])):
        grb_ra = float(df_grb_subset.iloc[n][1])
        grb_dec = float(df_grb_subset.iloc[n][2])
        
        dist = distance(frb_ra, grb_ra, frb_dec, grb_dec)
        dist_list.append(dist)
        
    min_dist_val = min(dist_list)
    grb_index = dist_list.index(min_dist_val)
    dist_s_no.append([m, grb_index, min_dist_val])
    


In [12]:
dist_s_no

[[0, 760, 2.596752510596917],
 [1, 291, 2.48303805797005],
 [2, 167, 3.1081312643501358],
 [3, 1362, 0.9603655074490652],
 [4, 1030, 0.932885260827942],
 [5, 60, 2.83975721994394],
 [6, 928, 1.637962389137355],
 [7, 1235, 2.036364428218165],
 [8, 934, 2.3771608506085227],
 [9, 787, 4.587342672335706],
 [10, 1051, 3.8748614394086935],
 [11, 553, 5.507600542197007],
 [12, 914, 0.9427335188259793],
 [13, 817, 3.2415040159324184],
 [14, 539, 1.5572456394462606],
 [15, 77, 2.8773264333407678],
 [16, 526, 1.1687445556391234],
 [17, 227, 4.2842627662389265],
 [18, 1252, 3.311576633522311],
 [19, 1011, 3.4575330214074858],
 [20, 218, 3.5333933457680398],
 [21, 1235, 1.887133121445595],
 [22, 472, 5.435066760828109],
 [23, 817, 3.801766627331521],
 [24, 665, 1.82989389341017],
 [25, 415, 3.7673664632263817],
 [26, 539, 1.5493414453399126],
 [27, 539, 1.5493414453399126],
 [28, 539, 1.5490013916320458],
 [29, 539, 1.5490013916320458],
 [30, 539, 1.5322133686184414],
 [31, 539, 1.5322133686184414

In [47]:
dist_less_than_zero_group = []  # List of frb/grb pairs at distance less than 1 unit.
grb_serial_no = []              # List of grb serial numbers.

for num1 in np.arange(0, len(dist_s_no)):
    grb_serial_no.append(dist_s_no[num1][1])
    if 0<dist_s_no[num1][2]<1:
        dist_less_than_zero_group.append(dist_s_no[num1])

print('There are', len(dist_less_than_zero_group), 'pair of frb/grb source at distance less than 1 unit')

There are 89 pair of frb/grb source at distance less than 1 unit


In [74]:
grb_code_no = []
grb_freq_val = []

for num2 in np.arange(0, len(grb_serial_no)):
    serial_no = grb_serial_no[num2]
    freq = grb_serial_no.count(serial_no)
    if grb_serial_no[num2] not in grb_code_no:
        grb_code_no.append(serial_no)
        grb_freq_val.append(freq)
        
        
# Now we pair up grb reference serial no and its frequence of pairing up with frb sources

grb_code_freq = []
for num3 in np.arange(0, len(grb_code_no)):
    grb_code_freq.append([grb_code_no[num3], grb_freq_val[num3]])

grb_code_freq

[[760, 1],
 [291, 1],
 [167, 1],
 [1362, 1],
 [1030, 1],
 [60, 1],
 [928, 1],
 [1235, 2],
 [934, 1],
 [787, 1],
 [1051, 1],
 [553, 1],
 [914, 1],
 [817, 2],
 [539, 23],
 [77, 1],
 [526, 2],
 [227, 2],
 [1252, 1],
 [1011, 1],
 [218, 3],
 [472, 2],
 [665, 1],
 [415, 1],
 [1365, 1],
 [397, 1],
 [340, 1],
 [464, 1],
 [696, 1],
 [496, 1],
 [407, 1],
 [47, 1],
 [297, 1],
 [306, 2],
 [466, 1],
 [658, 2],
 [1009, 1],
 [1158, 5],
 [250, 1],
 [30, 1],
 [56, 1],
 [364, 1],
 [1234, 1],
 [452, 2],
 [1173, 1],
 [19, 3],
 [543, 1],
 [1264, 1],
 [776, 1],
 [1313, 3],
 [132, 1],
 [1000, 1],
 [572, 1],
 [1103, 1],
 [1328, 2],
 [897, 1],
 [390, 1],
 [80, 2],
 [873, 1],
 [417, 1],
 [744, 1],
 [512, 1],
 [1096, 1],
 [338, 1],
 [585, 1],
 [1202, 2],
 [382, 1],
 [943, 2],
 [518, 1],
 [521, 2],
 [630, 1],
 [809, 8],
 [1052, 2],
 [463, 1],
 [138, 1],
 [349, 5],
 [1148, 3],
 [1374, 17],
 [140, 3],
 [1002, 1],
 [310, 3],
 [482, 1],
 [346, 3],
 [285, 4],
 [1211, 3],
 [1053, 10],
 [802, 7],
 [318, 1],
 [1152, 4],
